In [ ]:
import pandas as pd
import h5py as h5
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM ,Dropout , Dense, Bidirectional , convolutional
from keras.optimizers import Adam
from keras.layers.core import Reshape
import tensorflow
# from tensorflow.python.keras.layers import Dense , LSTM ,Dropout
# from tensorflow.python.keras import Sequential
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
new_df = pd.read_pickle(r"/content/drive/My Drive/pickle_dataframe")
print("len(new_df):", len(new_df),"\n")

len(new_df): 2052 



In [ ]:
print("len(new_df) with <50:", len(new_df),"\n")

new_df = new_df[new_df["c4_coord"].map(len)>=50].reset_index(drop = True)
print("len(new_df):", len(new_df),"\n")
new_df.head()

len(new_df) with <50: 2052 

len(new_df): 734 



,pdb_id_chain,res_code,res_number,dihedral,sin_dihedral,cos_dihedral,mean_of_x_pos,mean_of_y_pos,mean_of_z_pos,secondary_structure,p_coord,c5_coord,o5_coord,c4_coord,c3_coord,o3_coord,distance_by_c4,distance_by_mean,7_nearest_nuclt
0,5FLX_z_0,"[[-1, 1, -1, -1], [-1, -1, -1, 1], [-1, -1, 1,...","[45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 5...","[[9999.0, 169.22296295196648, 57.0660917772113...","[[9999.0, -0.41053466672835076, 0.494643000463...","[[9999.0, 0.9118449908916767, 0.86909625594181...","[-78.81483205159505, -76.81516774495442, -72.9...","[-48.50666745503744, -50.564666748046875, -53....","[81.61349995930989, 76.48966725667317, 73.1131...",[9999],"[[-79.435, -49.293, 84.352], [-78.858, -50.468...","[[-79.421, -47.917, 82.116], [-77.369, -49.755...","[[-78.777, -48.852, 82.97], [-77.556, -50.594,...","[[-78.501, -47.462, 81.011], [-75.941, -49.8, ...","[[-77.932, -48.566, 80.133], [-75.406, -51.182...","[[-78.823, -48.95, 79.099], [-75.761, -51.589,...",NaN,NaN,NaN
1,5FLX_z_1,"[[-1, -1, 1, -1], [1, -1, -1, -1], [-1, -1, -1...","[224, 225, 226, 227, 228, 229, 230, 231, 232, ...","[[72.25684171766095, 174.84430592493814, 54.94...","[[-0.00021068509414407045, -0.8842486283271339...","[[-0.9999999778058953, 0.46701644864135383, -0...","[-21.214666684468586, -22.792500178019207, -25...","[65.09783426920573, 59.435333887736, 54.262666...","[145.9863306681315, 146.67449951171875, 144.69...",[],"[[-20.061, 66.817, 143.971], [-20.926, 61.619,...","[[-20.742, 65.91, 146.336], [-22.384, 59.915, ...","[[-21.026, 66.01, 144.949], [-22.102, 60.551, ...","[[-21.852, 65.21, 147.079], [-23.763, 59.304, ...","[[-22.235, 63.832, 146.56], [-24.029, 58.254, ...","[[-21.372, 62.808, 147.023], [-23.551, 56.969,...",NaN,NaN,NaN
2,6D9J_4,"[[1, -1, -1, -1], [1, -1, -1, -1], [1, -1, -1,...","[6031, 6032, 6033, 6034, 6035, 6036, 6037, 603...","[[9999.0, -170.41386381061957, 126.92280212360...","[[9999.0, -0.6945976887212898, 0.9518134879812...","[[9999.0, 0.7193983950656564, 0.30667749199616...","[105.04666519165039, 105.81650034586589, 106.8...","[155.67733256022134, 150.70750172932944, 144.7...","[252.07866923014322, 255.80433146158853, 258.0...",[9999],"[[106.561, 157.292, 249.912], [105.453, 153.89...","[[105.199, 156.301, 251.938], [105.939, 151.30...","[[105.348, 156.452, 250.511], [105.016, 152.36...","[[104.13, 155.284, 252.282], [105.631, 150.048...","[[104.556, 154.168, 253.231], [106.205, 148.75...","[[104.486, 154.567, 254.598], [106.655, 147.88...",NaN,NaN,NaN
3,6GZ4_Bw,"[[-1, 1, -1, -1], [-1, -1, 1, -1], [-1, -1, 1,...","[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...","[[9999.0, 179.02521853832772, 52.8037271686045...","[[9999.0, 0.04554695352216948, 0.5674024798942...","[[9999.0, -0.9989621989969637, -0.823440602478...","[148.56733194986978, 150.02950286865234, 152.9...","[189.2875010172526, 186.70516713460287, 182.24...","[172.49516805013022, 167.53200022379556, 165.1...",[9999],"[[147.91, 188.352, 175.118], [148.253, 187.285...","[[148.036, 189.963, 173.046], [149.517, 187.56...","[[148.64, 188.946, 173.832], [149.436, 186.936...","[[148.927, 190.376, 171.901], [150.863, 187.34...","[[149.424, 189.245, 171.018], [151.338, 185.90...","[[148.467, 188.843, 170.056], [150.77, 185.192...",NaN,NaN,NaN
4,6ZMI_L5_0,"[[-1, -1, 1, -1], [-1, 1, -1, -1], [-1, -1, 1,...","[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...","[[9999.0, -145.19073303699454, 150.70224238054...","[[9999.0, -0.6268245119575182, -0.094065715280...","[[9999.0, 0.7791604656354294, 0.99556599038372...","[156.9134953816732, 152.0554987589518, 150.321...","[255.98050181070963, 258.78916422526044, 262.4...","[288.8191630045573, 286.55299377441406, 282.07...",[9999],"[[159.775, 254.908, 288.58], [153.694, 256.447...","[[157.218, 255.439, 288.718], [152.017, 258.32...","[[158.443, 255.219, 289.394], [152.95, 257.829...","[[156.368, 256.463, 289.425], [151.798, 259.80...","[[155.443, 257.269, 288.53], [151.597, 260.275...","[[154.234, 256.585, 288.268], [150.277, 260.05.

In [ ]:
##check res_number and convert from string to integer
l=[]
for i in range(len(new_df)):
  new_df['res_number'][i] = new_df['res_number'][i].astype(np.int)
  for j in range(len(new_df['res_number'][i])-1):
    if new_df['res_number'][i][j+1] != new_df['res_number'][i][j]+1:
      l.append(i)
      break
new_df = new_df.drop(l).reset_index(drop = True) 

In [ ]:
#@title def rotation 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d 
import math 

def x_y_z_genarator(coord):
    x = []
    y = []
    z = []
    for i in range(len(coord)):
        x.append(coord[i][0])
        y.append(coord[i][1])
        z.append(coord[i][2])
    return x, y, z

def get_rotation_Matrix(theta, axis):
    if axis == 'x':
        return np.array([[1, 0, 0],
                         [0, np.cos(theta), -1*np.sin(theta)],
                         [0, np.sin(theta), np.cos(theta)]],
                         dtype= np.float64)
    elif axis == 'y':
        return np.array([[np.cos(theta), 0, np.sin(theta)],
                         [0, 1, 0],
                         [-1*np.sin(theta), 0, np.cos(theta)]],
                         dtype= np.float64)
    elif axis == 'z':
        return np.array([[np.cos(theta), -1*np.sin(theta), 0],
                         [np.sin(theta), np.cos(theta), 0],
                         [0, 0, 1]],
                         dtype= np.float64)
    else:
        return None
    
def get_Angle_between_Vectors(v1,v2):
    return np.arccos((np.dot(v1,v2)) / (np.linalg.norm(v1)*np.linalg.norm(v2)))

def reflection_matrix(plane):
    if plane == "xz":
        return np.array([[1, 0, 0],
                         [0, -1, 0],
                         [0, 0, 1]            
                        ])
    else:
        return None
    
# for point 1 & 2 & 3 & 4
# [0,0,0],[0,0,z],[x,0,z],y>0
def standard_coordination(coordinates ):
    size = len(coordinates)
    
    for i in range(len(coordinates)-1, -1, -1):
        coordinates[i][0]  = coordinates[i][0] - coordinates[8][0]
        coordinates[i][1]  = coordinates[i][1] - coordinates[8][1] 
        coordinates[i][2]  = coordinates[i][2] - coordinates[8][2]
    vec_11 = [0, 0, 1]
    vec_12 = [0, coordinates[1][1], coordinates[1][2]]
#     print(coordinates[0],coordinates[1],coordinates[2])
    angle_1 = get_Angle_between_Vectors(vec_11,vec_12)
    
    if coordinates[1][1] > 0 :
        angle_1 = math.radians(360)-angle_1
#     print("angle_1 :", angle_1)
    rot_1 = get_rotation_Matrix(angle_1,"x")
    coordinates_2 =np.dot(coordinates, rot_1)
    vec_21 = [0, 0, 1]
    vec_22 = coordinates_2[1] 
    angle_2 = get_Angle_between_Vectors(vec_21, vec_22)
    
    if coordinates_2[1][0] < 0 :
        angle_2 = math.radians(360)-angle_2
#     print("angle_2 :", angle_2)
    rot_2 = get_rotation_Matrix(angle_2, "y")
    coordinates_3 = np.dot(coordinates_2, rot_2)
    
    vec_31 = [1, 0, 0]
    vec_32 = [coordinates_3[2][0], coordinates_3[2][1], 0]
    angle_3 = get_Angle_between_Vectors(vec_31, vec_32)
    
    if coordinates_3[2][1] < 0 :
        angle_3 = math.radians(360)-angle_3
#     print("angle_3 :", angle_3)
    rot_3 = get_rotation_Matrix(angle_3, "z")
    coordinates_4 = np.dot(coordinates_3, rot_3)
    
    if coordinates_4[3][1] < 0:
        coordinates_5 = np.dot(coordinates_4,reflection_matrix("xz"))
        return coordinates_5
    
    else:
        return coordinates_4
    
# coord = np.array([[0, 0, 0],[0, 1, 0],[0, 1, 1],[-1, 1, 1]])
# len(real_coords)
# coord = real_coords.reshape(10,3)

# new = standard_coordination(coord)

# print("rotated coordination error:", sum((ans-new.flatten())**2))

In [ ]:
#bayad har 15 taii ro bedim b in tabe ta standardesh kone
##for 15:coord[44] & for 7:coord[20]
def standard_coordination(coordinates ):
    coord=[]
    size = len(coordinates)
    for i in range(7):
      for j in range(6):
        coord.append(coordinates[i][j])
    for i in range(len(coord)-1, -1, -1):
        coord[i][0]  = coord[i][0] - coord[20][0]
        coord[i][1]  = coord[i][1] - coord[20][1] 
        coord[i][2]  = coord[i][2] - coord[20][2]
    vec_11 = [0, 0, 1]
    vec_12 = [0, coord[21][1], coord[21][2]]
#     print(coordinates[0],coordinates[1],coordinates[2])
    angle_1 = get_Angle_between_Vectors(vec_11,vec_12)
    
    if coord[21][1] > 0 :
        angle_1 = math.radians(360)-angle_1
#     print("angle_1 :", angle_1)
    rot_1 = get_rotation_Matrix(angle_1,"x")
    coordinates_2 =np.dot(coord, rot_1)
    vec_21 = [0, 0, 1]
    vec_22 = coordinates_2[21] 
    angle_2 = get_Angle_between_Vectors(vec_21, vec_22)
    
    if coordinates_2[21][0] < 0 :
        angle_2 = math.radians(360)-angle_2
#     print("angle_2 :", angle_2)
    rot_2 = get_rotation_Matrix(angle_2, "y")
    coordinates_3 = np.dot(coordinates_2, rot_2)
    
    vec_31 = [1, 0, 0]
    vec_32 = [coordinates_3[22][0], coordinates_3[22][1], 0]
    angle_3 = get_Angle_between_Vectors(vec_31, vec_32)
    
    if coordinates_3[22][1] < 0 :
        angle_3 = math.radians(360)-angle_3
#     print("angle_3 :", angle_3)
    rot_3 = get_rotation_Matrix(angle_3, "z")
    coordinates_4 = np.dot(coordinates_3, rot_3)
    
    if coordinates_4[23][1] < 0:
        coordinates_5 = np.dot(coordinates_4,reflection_matrix("xz"))
        return coordinates_5
    
    else:
        return coordinates_4

In [ ]:
def distance_finder(one,two) :      
  [x1,y1,z1] = one # first coordinates     
  [x2,y2,z2] = two # second coordinates      
  dist = (((x2-x1)**2)+((y2-y1)**2)+((z2-z1)**2))**(1/2)     
  return dist

In [ ]:
list(new_df)

['pdb_id_chain',
 'res_code',
 'res_number',
 'dihedral',
 'sin_dihedral',
 'cos_dihedral',
 'mean_of_x_pos',
 'mean_of_y_pos',
 'mean_of_z_pos',
 'secondary_structure',
 'p_coord',
 'c5_coord',
 'o5_coord',
 'c4_coord',
 'c3_coord',
 'o3_coord',
 'distance_by_c4',
 'distance_by_mean',
 '7_nearest_nuclt']

In [ ]:
def main(new_df,nearest):
  x=[]
  y=[]
  for i in range(len(new_df)-635):
    for j in range(len(new_df['c4_coord'][i])):
      dist_list=[]
      for k in range(len(new_df['c4_coord'][i])):
        dist_list.append(distance_finder(new_df['c4_coord'][i][j],new_df['c4_coord'][i][k]))
      dist_list = np.array(dist_list)
      indx = dist_list.argsort()[:nearest]
      l=[]
      for n in indx:
        co=[]
        co.append((new_df['p_coord'][i][n]))
        co.append(new_df['c5_coord'][i][n])
        co.append(new_df['o5_coord'][i][n])
        co.append(new_df['c4_coord'][i][n])
        co.append(new_df['c3_coord'][i][n])
        co.append(new_df['o3_coord'][i][n])
        # co.append(new_df['mean_of_x_pos'][i][n])
        # co.append(new_df['mean_of_y_pos'][i][n])
        # co.append(new_df['mean_of_z_pos'][i][n])
        # co.append(new_df['dihedral'][i][n])
        # co.append(new_df['sin_dihedral'][i][n])
        # co.append(new_df['cos_dihedral'][i][n])
        l.append(co)

      x.append(l)
      y.append(new_df['res_code'][i][j])
  return x , y
# new_df['c4_coord'][0]

In [ ]:
x , y =main(new_df, 7)

In [ ]:
len(x)

25392

In [ ]:
##baad az inke stanard kardim bayad bedim b in tabe ta mokhtasate har 6 atom to baham begire
def standard_to_6(std):
  coordinates=[]
  for i in range(0,len(std),6):
    coord=[]
    for j in range(i,i+6):
      coord.append(std[j])
    coordinates.append(coord)
  return coordinates
def y_to_label(y):
  y1=[]
  for i in range(len(y)):
    if y[i] == [1,-1,-1,-1]:
      y1.append(0)
    if y[i] == [-1,1,-1,-1]:
      y1.append(1)
    if y[i] == [-1,-1,1,-1]:
      y1.append(2)
    if y[i] == [-1,-1,-1,1]:
      y1.append(3) 
  return y1

In [ ]:
x_std=[]
for i in range(len(x)):
  x_std.append(standard_to_6(standard_coordination(x[i])))

In [ ]:
data=[]
for i in range(len(x_std)):
  d=[]
  for j in range(7):
    for k in range(6):
      for s in range(3):
        d.append(x_std[i][j][k][s])
  data.append(d)
df = pd.DataFrame(data) 

In [ ]:
from sklearn.cluster import KMeans 

clusters = 4
  
kmeans = KMeans(n_clusters = clusters) 
kmeans.fit(df) 
  
l=(kmeans.labels_)
l1=(y_to_label(y))


In [ ]:
m=0
for i in range(len(l1)):
  if l[i] == l1[i]:
    m=m+1
print(m/735)
##cluster kardan ba standard coordinate ha faydeii nadare

7.253061224489796


In [ ]:
d=[]
for i in range(len(l)):
  if l[i] == 3:
    d.append(i)

q , w,e,r=0 , 0,0,0
for i in range(len(d)):
    if l1[d[i]] == 0:
      q = q+1
    if l1[d[i]] == 1:
      w = w+1
    if l1[d[i]] == 2:
      e = e+1
    if l1[d[i]] == 3:
      r = r+1
q,w,e,r

(231, 217, 277, 183)

In [ ]:
sef , sef1, sef2, sef3= 0,0,0,0
for i in range(len(l1)):
  if l1[i] == 0:
    sef=sef+1
  if l1[i] == 1:
    sef1=sef1+1
  if l1[i] == 2:
    sef2=sef2+1
  if l1[i] == 3:
    sef3=sef3+1

In [ ]:
sef ,sef1,sef2,sef3

(5203, 4474, 5419, 4212)

In [ ]:
q,w,e,r

##dar sar resid neveshte shode natayeje clustering ha

In [ ]:
from numpy import nan


## clustering for dihedral

In [ ]:
def main(new_df,nearest):
  x=[]
  y=[]
  for i in range(len(new_df)-635):
    for j in range(len(new_df['c4_coord'][i])):
      dist_list=[]
      for k in range(len(new_df['c4_coord'][i])):
        dist_list.append(distance_finder(new_df['c4_coord'][i][j],new_df['c4_coord'][i][k]))
      dist_list = np.array(dist_list)
      indx = dist_list.argsort()[:nearest]
      l=[]
      for n in indx:
        co=[]
        # co.append((new_df['p_coord'][i][n]))
        # co.append(new_df['c5_coord'][i][n])
        # co.append(new_df['o5_coord'][i][n])
        # co.append(new_df['c4_coord'][i][n])
        # co.append(new_df['c3_coord'][i][n])
        # co.append(new_df['o3_coord'][i][n])
        # co.append(new_df['mean_of_x_pos'][i][n])
        # co.append(new_df['mean_of_y_pos'][i][n])
        # co.append(new_df['mean_of_z_pos'][i][n])
        # co.append(new_df['dihedral'][i][n])
        co.append(new_df['sin_dihedral'][i][n])
        co.append(new_df['cos_dihedral'][i][n])
        l.append(co)
      x.append(l)
      y.append(new_df['res_code'][i][j])
  return x , y

x , y =main(new_df, 7)

data=[]
for i in range(len(x)):
  d=[]
  for j in range(7):
    for k in range(2):
      for f in range(6):
        d.append(x[i][j][k][f])
  data.append(d)
df = pd.DataFrame(data) 

NameError: ignored

In [ ]:
df.isnull().sum().sum()

0

In [ ]:
len(x)

19308

In [ ]:
df.fillna(0.5, inplace=True)

In [ ]:
df.replace([9999],0.5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179
0,-0.690781,-0.051029,0.346730,0.871152,0.414979,0.848227,-0.723064,-0.998697,-0.937965,0.491013,-0.909831,0.529632,-0.396775,0.308647,-0.409842,0.992652,0.984453,0.897568,0.917916,-0.951177,0.912156,-0.121000,0.175646,0.440876,-0.936284,-0.928855,-0.960765,-0.235252,-0.408052,-0.271811,-0.351243,-0.370443,0.277363,-0.971934,-0.912959,0.962351,-0.733392,0.325277,0.983582,-0.974854,...,-0.507066,0.097689,0.107267,0.936957,0.186930,0.702292,0.090941,0.986200,0.590685,-0.821277,-0.982373,-0.711889,0.995856,-0.165558,-0.806902,0.570530,0.905598,-0.362739,0.720239,0.220713,0.604416,0.233201,0.424138,0.931891,0.693726,-0.975339,0.796669,-0.972428,0.040391,0.989149,-0.302648,-0.999986,-0.951794,0.489864,0.999184,0.146915,0.953102,-0.005269,0.306737,-0.871799
1,0.500000,-0.151145,0.507650,0.348377,0.504309,0.682230,0.500000,0.988512,-0.861563,0.937354,-0.863523,0.731138,-0.251580,-0.755057,-0.999408,0.921833,0.982473,-0.920803,-0.967837,-0.655659,0.034390,-0.387587,0.186404,0.390027,0.178558,0.882033,0.999654,-0.689405,0.506729,0.967601,-0.983929,0.471188,-0.026295,0.724376,-0.862105,-0.252484,0.459950,-0.109824,-0.970865,-0.418054,...,-0.552872,0.995598,0.500000,0.500000,0.998217,-0.943385,-0.696577,0.761570,-0.186292,-0.740262,-0.059697,0.331699,0.717482,-0.648083,-0.982494,0.672318,-0.534623,0.383228,0.613730,0.927468,-0.998508,0.607556,-0.845091,0.923654,0.789516,0.373903,0.054608,-0.794277,0.822061,0.953666,-0.602788,-0.861052,0.311531,-0.937120,0.569400,0.300868,0.797902,0.508518,0.950236,0.349007
2,0.294679,-0.853300,0.636201,-0.928039,0.225578,-0.563983,0.955596,-0.521421,0.771523,-0.372484,0.974225,0.825786,-0.384443,0.065554,-0.621246,0.970552,0.743174,-0.021906,0.923149,0.997849,0.783615,0.240891,-0.669098,-0.999760,0.759826,0.319089,-0.985237,0.016816,-0.998564,0.842183,0.650126,0.947725,0.171198,-0.999859,-0.053580,0.539192,0.898553,0.803775,0.988164,-0.591530,...,-0.372728,-0.997806,0.767705,0.032760,-0.798993,-0.573638,-0.617341,-0.961394,0.768024,-0.431293,-0.601341,-0.819109,0.786695,-0.275176,0.640421,0.902212,0.097919,-0.902095,0.790901,-0.184622,-0.250881,-0.827632,0.995194,0.431537,0.611945,-0.982810,0.968018,0.561270,0.365286,-0.890286,-0.762529,0.067357,0.786436,-0.829049,0.930895,-0.455401,0.646954,0.997729,0.617671,-0.559176
3,0.500000,-0.375817,-0.241051,-0.830461,-0.769629,0.926030,0.500000,0.926694,0.970512,-0.557076,0.638492,-0.377449,0.886368,0.005763,-0.971837,-0.752136,-0.985426,0.976281,0.462982,-0.999983,0.235653,-0.659008,-0.170106,-0.216505,-0.305165,0.996731,-0.798216,-0.451254,0.500000,0.500000,-0.952300,0.080793,-0.602371,0.892395,0.500000,0.500000,0.995045,-0.891339,-0.956274,-0.432943,...,0.296052,-0.975325,-0.924268,-0.945615,0.941730,0.263971,-0.995792,-0.299502,-0.860864,0.926076,0.336369,0.964531,-0.091638,0.954096,0.508834,0.377338,0.246591,-0.117214,0.236410,-0.359777,-0.835801,0.656522,-0.969120,-0.993107,0.971654,0.933038,-0.549033,-0.754307,0.459084,0.975647,0.981944,0.171168,-0.015595,0.999991,-0.888393,0.219348,-0.189171,-0.985242,0.999878,0.004235
4,0.500000,0.923166,-0.543175,0.681975,0.929209,-0.470831,0.500000,-0.384401,-0.839620,-0.731375,-0.369554,-0.882223,0.857958,-0.649202,0.269820,0.969678,0.987850,0.475507,0.513719,-0.760616,0.962911,0.244385,-0.155413,-0.879712,0.778400,0.891316,0.016732,-0.942750,0.744873,0.713221,-0.627768,-0.453382,0.999860,0.333501,-0.667206,-0.700939,-0.574592,0.948324,-0.411632,0.916956,...,0.913106,0.974161,0.851920,-0.721981,-0.766315,-0.297925,0.641493,-0.165729,-0.988131,0.999183,-0.642465,0.954589,0.767129,-0.986171,-0.153616,0.040409,-0.966201,0.035633,-0.811437,0.972692,0.747396,0.944065,0.257790,0.999365,-0.584440,0.232100,0.664379,-0.329758,0.432338,0.895083,-0.999977,-0

In [ ]:
from sklearn.cluster import KMeans 

clusters = 4
  
kmeans = KMeans(n_clusters = clusters) 
kmeans.fit(df) 
  
l=(kmeans.labels_)
l1=(y_to_label(y))

In [ ]:
d=[]
for i in range(len(l)):
  if l[i] == 3:
    d.append(i)

q , w,e,r=0 , 0,0,0
for i in range(len(d)):
    if l1[d[i]] == 0:
      q = q+1
    if l1[d[i]] == 1:
      w = w+1
    if l1[d[i]] == 2:
      e = e+1
    if l1[d[i]] == 3:
      r = r+1
q,w,e,r

(6, 12, 10, 10)

In [ ]:
q,w,e,r


(49, 50, 94, 52)

## clustering with sin , cos , standard cood

In [ ]:
def main(new_df,nearest):
  x=[]
  y=[]
  for i in range(len(new_df)-600):
    for j in range(len(new_df['c4_coord'][i])):
      dist_list=[]
      for k in range(len(new_df['c4_coord'][i])):
        dist_list.append(distance_finder(new_df['c4_coord'][i][j],new_df['c4_coord'][i][k]))
      dist_list = np.array(dist_list)
      indx = dist_list.argsort()[:nearest]
      l=[]
      for n in indx:
        co=[]
        co.append((new_df['p_coord'][i][n]))
        co.append(new_df['c5_coord'][i][n])
        co.append(new_df['o5_coord'][i][n])
        co.append(new_df['c4_coord'][i][n])
        co.append(new_df['c3_coord'][i][n])
        co.append(new_df['o3_coord'][i][n])
        # co.append(new_df['mean_of_x_pos'][i][n])
        # co.append(new_df['mean_of_y_pos'][i][n])
        # co.append(new_df['mean_of_z_pos'][i][n])
        # co.append(new_df['dihedral'][i][n])
        co.append(new_df['sin_dihedral'][i][n])
        co.append(new_df['cos_dihedral'][i][n])
        l.append(co)
      x.append(l)
      y.append(new_df['res_code'][i][j])
  return x , y

x , y =main(new_df, 15)



In [ ]:
from sklearn.preprocessing import normalize
#bayad har 15 taii ro bedim b in tabe ta standardesh kone
def standard_coordination_plus_dihed(coordinates ):
    l=[]
    coord=[]
    dihed=[]
    size = len(coordinates)
    for i in range(15):
      for j in range(6):
        coord.append(coordinates[i][j])
      for j in range(6,8):
        dihed.append(coordinates[i][j])
    for i in range(len(coord)-1, -1, -1):
        coord[i][0]  = coord[i][0] - coord[44][0]
        coord[i][1]  = coord[i][1] - coord[44][1] 
        coord[i][2]  = coord[i][2] - coord[44][2]
    vec_11 = [0, 0, 1]
    vec_12 = [0, coord[45][1], coord[45][2]]
#     print(coordinates[0],coordinates[1],coordinates[2])
    angle_1 = get_Angle_between_Vectors(vec_11,vec_12)
    
    if coord[45][1] > 0 :
        angle_1 = math.radians(360)-angle_1
#     print("angle_1 :", angle_1)
    rot_1 = get_rotation_Matrix(angle_1,"x")
    coordinates_2 =np.dot(coord, rot_1)
    vec_21 = [0, 0, 1]
    vec_22 = coordinates_2[45] 
    angle_2 = get_Angle_between_Vectors(vec_21, vec_22)
    
    if coordinates_2[45][0] < 0 :
        angle_2 = math.radians(360)-angle_2
#     print("angle_2 :", angle_2)
    rot_2 = get_rotation_Matrix(angle_2, "y")
    coordinates_3 = np.dot(coordinates_2, rot_2)
    
    vec_31 = [1, 0, 0]
    vec_32 = [coordinates_3[46][0], coordinates_3[46][1], 0]
    angle_3 = get_Angle_between_Vectors(vec_31, vec_32)
    
    if coordinates_3[46][1] < 0 :
        angle_3 = math.radians(360)-angle_3
#     print("angle_3 :", angle_3)
    rot_3 = get_rotation_Matrix(angle_3, "z")
    coordinates_4 = np.dot(coordinates_3, rot_3)
    
    if coordinates_4[47][1] < 0:
        coordinates_5 = np.dot(coordinates_4,reflection_matrix("xz"))

        # coordinates_5 = normalize( coordinates_5)
        l.append(standard_to_6(coordinates_5))
        k=0
        for i in range(0,len(dihed),2):
          k=int(i/2)
          l[0][k].append(dihed[i])
          l[0][k].append(dihed[i+1])
        return l[0]
    
    else:
        # coordinates_4 = normalize( coordinates_4)
        l.append(standard_to_6(coordinates_4))
        k=0
        for i in range(0,30,2):
          k=int(i/2)
          l[0][k].append(dihed[i])
          l[0][k].append(dihed[i+1])
        return l[0]

In [ ]:
##standard coord
x_std=[]
for i in range(len(x)):
  x_std.append(standard_coordination_plus_dihed(x[i]))

In [ ]:
##dade baraye x haii k standard va normalize shodan
data=[]
for i in range(len(x_std)):
  d=[]
  for j in range(15):
    for k in range(8):
      if k <6:
        for q in range(3):
          d.append(x_std[i][j][k][q])
      else:
        for e in range(6):
          d.append(x_std[i][j][k][e])
  data.append(d)
df2 = pd.DataFrame(data) 

In [ ]:
##dade baraye x haii k standard va normalize nashodan
data=[]
for i in range(len(x)):
  d=[]
  for j in range(15):
    for k in range(8):
      if k <6:
        for q in range(3):
          d.append(x[i][j][k][q])
      else:
        for e in range(6):
          d.append(x[i][j][k][e])
  data.append(d)
df = pd.DataFrame(data) 

In [ ]:
#example for get unit vector
distance = [mouse.x - player.x, mouse.y - player.y]
norm = math.sqrt(distance[0] ** 2 + distance[1] ** 2)
direction = [distance[0] / norm, distance[1] / norm]


In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449
0,-1.233,-2.963,12.276,0.074,-5.157,11.658,-0.021,-3.739,11.588,1.435,-5.599,12.143,1.550,-7.060,12.569,1.758,-7.959,11.491,-0.690781,-0.051029,0.346730,0.871152,0.414979,0.848227,-0.723064,-0.998697,-0.937965,0.491013,-0.909831,0.529632,83.938,-167.631,-100.134,84.357,-168.575,-97.729,0.000,0.000,0.000,0.752,...,0.619006,-0.522723,0.643720,0.585176,-0.474662,-0.401070,-0.785386,-0.852502,0.765261,-0.810907,-11.220,5.392,15.557,-12.745,6.669,13.845,0.000,0.000,0.000,-1.117,1.623,-1.427,-0.339,1.147,-2.641,-1.057,0.251,-3.472,-0.997602,-0.030350,-0.270989,0.981805,0.983608,-0.821388,0.069205,0.999539,0.962583,-0.189889,-0.180319,0.570370
1,5.554,7.723,-10.841,6.959,6.379,-9.087,6.605,6.595,-10.443,8.387,5.916,-8.971,8.643,4.435,-9.202,8.630,4.051,-10.566,-0.396775,0.308647,-0.409842,0.992652,0.984453,0.897568,0.917916,-0.951177,0.912156,-0.121000,0.175646,0.440876,62.629,-153.002,-97.021,64.393,-151.933,-98.627,0.000,0.000,0.000,1.552,...,0.983582,-0.974854,0.030189,-0.975037,0.679806,-0.945619,-0.180463,-0.222843,-0.999544,0.222044,-62.292,138.986,95.600,-64.723,139.601,96.355,0.000,0.000,0.000,-1.830,1.072,1.176,-3.282,0.633,1.212,-3.418,-0.755,1.483,0.423172,-0.662375,-0.968254,0.967060,0.641822,-0.068058,0.906049,0.749173,0.249970,-0.254549,0.766853,-0.997681
2,-282.396,553.988,316.734,-280.604,552.133,316.114,-280.936,553.514,316.267,-279.125,551.887,316.327,-278.711,550.471,316.744,-278.556,549.557,315.661,-0.986514,-0.965390,-0.628895,0.694533,0.912545,0.367939,0.163677,-0.260812,0.777490,-0.719461,0.408977,-0.929850,66.818,-150.669,-107.635,66.762,-153.188,-106.887,0.000,0.000,0.000,1.077,...,-0.968254,0.967060,0.641822,-0.068058,0.906049,0.749173,0.249970,-0.254549,0.766853,-0.997681,67.704,-144.931,-107.630,68.518,-142.468,-107.164,0.000,0.000,0.000,1.031,2.092,0.619,1.732,3.378,0.199,0.854,4.458,-0.045,-0.966620,-0.491006,-0.347719,0.994988,0.868024,0.446013,0.256214,-0.871156,-0.937599,0.099998,0.496521,0.895027
3,-61.958,149.355,91.574,-59.714,148.261,92.454,0.000,0.000,0.000,1.637,-1.455,1.056,0.917,-2.707,1.552,1.403,-3.902,0.947,0.474983,-0.952262,0.891381,0.478733,0.961967,-0.075206,-0.879995,0.305283,-0.453255,-0.877960,0.273167,-0.997168,83.938,-167.631,-100.134,84.357,-168.575,-97.729,0.000,0.000,0.000,0.752,...,-0.270989,0.981805,0.983608,-0.821388,0.069205,0.999539,0.962583,-0.189889,-0.180319,0.570370,-282.396,553.988,316.734,-280.604,552.133,316.114,-280.936,553.514,316.267,-279.125,551.887,316.327,-278.711,550.471,316.744,-278.556,549.557,315.661,-0.986514,-0.965390,-0.628895,0.694533,0.912545,0.367939,0.163677,-0.260812,0.777490,-0.719461,0.408977,-0.929850
4,-68.830,145.376,102.801,-68.305,143.835,104.850,-8.400,1.318,9.682,-7.956,0.253,11.796,-9.194,-0.626,11.880,-8.906,-2.015,11.741,0.107296,0.105161,0.970774,-0.888462,0.569977,-0.255630,0.994227,0.994455,-0.239996,0.458951,0.821661,0.966775,-16.403,-14.474,12.212,-14.407,-13.681,13.760,-69.229,150.435,102.367,-67.305,...,0.940608,0.278657,0.544609,0.523394,0.774563,0.630791,0.339495,0.960391,0.838690,0.852091,-282.396,553.988,316.734,-280.604,552.133,316.114,-280.936,553.514,316.267,-279.125,551.887,316.327,-278.711,550.471,316.744,-278.556,549.557,315.661,-0.986514,-0.965390,-0.628895,0.694533,0.912545,0.367939,0.163677,-0.260812,0.777490,-0.719461,0.408977,-0.929850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22024,24.761,-236.242,-37.096,25.689,-237.142,-34.848,0.000,0.000,0.000,1.875,-0.454,1.448

In [ ]:
df.isnull().sum().sum()

322

In [ ]:
df2.replace([9999],0.5)
df2.fillna(0.5, inplace=True)



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449
0,0.180337,0.859827,0.477677,0.180092,0.857962,0.481110,0.176855,0.859703,0.479201,0.176866,0.855693,0.486322,0.180582,0.853168,0.489382,0.180561,0.854332,0.487355,-0.690781,-0.051029,0.346730,0.871152,0.414979,0.848227,-0.723064,-0.998697,-0.937965,0.491013,-0.909831,0.529632,-0.373759,0.923789,-0.083172,-0.374711,0.925036,-0.062442,0.000000,0.000000,0.000000,-0.223438,...,0.720239,0.220713,0.604416,0.233201,0.424138,0.931891,0.693726,-0.975339,0.796669,-0.972428,0.588015,0.073044,0.805545,0.595410,0.051312,0.801782,0.596761,0.063644,0.799891,0.600648,0.045585,0.798213,0.606622,0.034375,0.794247,0.601337,0.023831,0.798640,0.040391,0.989149,-0.302648,-0.999986,-0.951794,0.489864,0.999184,0.146915,0.953102,-0.005269,0.306737,-0.871799
1,-0.455377,0.459926,-0.762299,-0.472671,0.449144,-0.758190,0.000000,0.000000,0.000000,-0.562356,-0.805290,0.187785,-0.843386,-0.452634,0.289522,-0.992056,-0.109348,0.062191,-0.396775,0.308647,-0.409842,0.992652,0.984453,0.897568,0.917916,-0.951177,0.912156,-0.121000,0.175646,0.440876,-0.905964,0.407633,-0.114301,-0.907593,0.404431,-0.112745,-0.906867,0.404908,-0.116795,-0.910197,...,0.090941,0.986200,0.590685,-0.821277,-0.982373,-0.711889,0.995856,-0.165558,-0.806902,0.570530,-0.967280,0.224976,-0.117279,-0.967261,0.226349,-0.114772,-0.258372,-0.936663,-0.236444,-0.233529,-0.946155,-0.224175,-0.234968,-0.943554,-0.233442,-0.244764,-0.943307,-0.224192,0.995383,-0.580594,-0.975407,0.338558,-0.231850,0.923556,-0.095980,-0.814193,-0.220410,-0.940945,0.972752,-0.383464
2,-0.660101,-0.205173,0.722614,-0.691340,-0.192502,0.696414,0.512686,0.180581,-0.839371,0.507165,0.185711,-0.841603,0.507211,0.191054,-0.840378,0.508605,0.187988,-0.840227,-0.986514,-0.965390,-0.628895,0.694533,0.912545,0.367939,0.163677,-0.260812,0.777490,-0.719461,0.408977,-0.929850,-0.637233,-0.324145,0.699188,-0.643306,-0.319250,0.695872,0.000000,0.000000,0.000000,-0.151362,...,0.039942,0.971750,-0.512620,0.995089,0.491928,0.433590,-0.999202,0.236012,0.858616,-0.098983,0.592338,-0.032848,-0.805020,0.593443,-0.029805,-0.804324,0.593208,-0.031725,-0.804424,0.592651,-0.028786,-0.804945,0.592730,-0.026708,-0.804958,0.593049,-0.025828,-0.804752,-0.936284,-0.928855,-0.960765,-0.235252,-0.408052,-0.271811,-0.351243,-0.370443,0.277363,-0.971934,-0.912959,0.962351
3,-0.966853,-0.122487,-0.224038,-0.968900,-0.120731,-0.216002,0.000000,0.000000,0.000000,0.362164,0.008830,0.932073,0.588919,-0.336613,0.734756,0.755964,-0.143785,0.638627,0.474983,-0.952262,0.891381,0.478733,0.961967,-0.075206,-0.879995,0.305283,-0.453255,-0.877960,0.273167,-0.997168,-0.737097,0.468467,0.487060,-0.732709,0.457903,0.503450,-0.737790,0.459583,0.494417,-0.735289,...,-0.270989,0.981805,0.983608,-0.821388,0.069205,0.999539,0.962583,-0.189889,-0.180319,0.570370,0.872343,-0.487036,-0.042593,0.883432,-0.468042,0.022042,0.437893,-0.782534,0.442596,0.410644,-0.758527,0.505973,0.435507,-0.741041,0.511070,0.462399,-0.714055,0.525655,0.753252,-0.929796,-0.876154,-0.990912,0.668241,0.665151,0.657732,0.368075,-0.482031,-0.134513,-0.743945,0.746709
4,0.207881,0.935646,-0.285224,0.198385,0.932280,-0.302485,0.206849,0.932434,-0.296276,0.200335,0.928373,-0.313033,0.201959,0.926942,-0.316215,0.192667,0.928280,-0.318082,0.107296,0.105161,0.970774,-0.888462,0.569977,-0.255630,0.994227,0.994455,-0.239996,0.458951,0.821661,0.966775,0.846771,0.323937,-0.421953,0.848855,0.318956,-0.421560,0.000000,0.000000,0.000000,0.836676,...,-0.270989,0.981805,0.983608,-0.821388,0.069205,0.999539,0.962583,-0.189889,-0.180319,0.570370,0.798625,0.406635,-0.443674,0.792667,0.415243,-0.446377,0.764604,0.404730,-0.501572,0.759287,0.411209,-0.504371,0.757130,0.410774,-0.507955,0.755623,0.414849,-0.506887,0.753252,-0.929796,-0.876154,-0.990

In [ ]:
from sklearn.cluster import KMeans 

clusters = 4
  
kmeans = KMeans(n_clusters = clusters) 
kmeans.fit(df2) 
  
l=(kmeans.labels_)
l1=(y_to_label(y))

In [ ]:
d=[]
for i in range(len(l)):
  if l[i] == 0:
    d.append(i)

q , w,e,r=0 , 0,0,0
for i in range(len(d)):
    if l1[d[i]] == 0:
      q = q+1
    if l1[d[i]] == 1:
      w = w+1
    if l1[d[i]] == 2:
      e = e+1
    if l1[d[i]] == 3:
      r = r+1
q,w,e,r

(5861, 5023, 6133, 4860)

In [ ]:
##faghat standard kardan coordinate ha
#bayad har 15 taii ro bedim b in tabe ta standardesh kone
def standard_coordination_plus_dihed1(coordinates ):
    l=[]
    coord=[]
    dihed=[]
    size = len(coordinates)
    for i in range(15):
      for j in range(6):
        coord.append(coordinates[i][j])
      for j in range(6,8):
        dihed.append(coordinates[i][j])
    for i in range(len(coord)-1, -1, -1):
        coord[i][0]  = coord[i][0] - coord[44][0]
        coord[i][1]  = coord[i][1] - coord[44][1] 
        coord[i][2]  = coord[i][2] - coord[44][2]
    vec_11 = [0, 0, 1]
    vec_12 = [0, coord[45][1], coord[45][2]]
#     print(coordinates[0],coordinates[1],coordinates[2])
    angle_1 = get_Angle_between_Vectors(vec_11,vec_12)
    
    if coord[45][1] > 0 :
        angle_1 = math.radians(360)-angle_1
#     print("angle_1 :", angle_1)
    rot_1 = get_rotation_Matrix(angle_1,"x")
    coordinates_2 =np.dot(coord, rot_1)
    vec_21 = [0, 0, 1]
    vec_22 = coordinates_2[45] 
    angle_2 = get_Angle_between_Vectors(vec_21, vec_22)
    
    if coordinates_2[45][0] < 0 :
        angle_2 = math.radians(360)-angle_2
#     print("angle_2 :", angle_2)
    rot_2 = get_rotation_Matrix(angle_2, "y")
    coordinates_3 = np.dot(coordinates_2, rot_2)
    
    vec_31 = [1, 0, 0]
    vec_32 = [coordinates_3[46][0], coordinates_3[46][1], 0]
    angle_3 = get_Angle_between_Vectors(vec_31, vec_32)
    
    if coordinates_3[46][1] < 0 :
        angle_3 = math.radians(360)-angle_3
#     print("angle_3 :", angle_3)
    rot_3 = get_rotation_Matrix(angle_3, "z")
    coordinates_4 = np.dot(coordinates_3, rot_3)
    
    if coordinates_4[47][1] < 0:
        coordinates_5 = np.dot(coordinates_4,reflection_matrix("xz"))

        l.append(standard_to_6(coordinates_5))
        k=0
        for i in range(0,len(dihed),2):
          k=int(i/2)
          l[0][k].append(dihed[i])
          l[0][k].append(dihed[i+1])
        return l[0]
    
    else:

        l.append(standard_to_6(coordinates_4))
        k=0
        for i in range(0,30,2):
          k=int(i/2)
          l[0][k].append(dihed[i])
          l[0][k].append(dihed[i+1])
        return l[0]

In [ ]:
##standard coord
x_std=[]
for i in range(len(x)):
  x_std.append(standard_coordination_plus_dihed1(x[i]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
##dade baraye x haii k standard shodan
data=[]
for i in range(len(x_std)):
  d=[]
  for j in range(15):
    for k in range(8):
      if k <6:
        for q in range(3):
          d.append(x_std[i][j][k][q])
      else:
        for e in range(6):
          d.append(x_std[i][j][k][e])
  data.append(d)
df3 = pd.DataFrame(data) 

In [ ]:
df3.replace([9999],0.5)
df3.fillna(0.5, inplace=True)

In [ ]:
from sklearn.cluster import KMeans 

clusters = 4
  
kmeans = KMeans(n_clusters = clusters) 
kmeans.fit(df3) 
  
l=(kmeans.labels_)
l1=(y_to_label(y))

In [ ]:
d=[]
for i in range(len(l)):
  if l[i] == 0:
    d.append(i)

q , w,e,r=0 , 0,0,0
for i in range(len(d)):
    if l1[d[i]] == 0:
      q = q+1
    if l1[d[i]] == 1:
      w = w+1
    if l1[d[i]] == 2:
      e = e+1
    if l1[d[i]] == 3:
      r = r+1
q,w,e,r

NameError: ignored

In [ ]:

df1=df.transpose()
df1.replace([9999],0.5)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,21989,21990,21991,21992,21993,21994,21995,21996,21997,21998,21999,22000,22001,22002,22003,22004,22005,22006,22007,22008,22009,22010,22011,22012,22013,22014,22015,22016,22017,22018,22019,22020,22021,22022,22023,22024,22025,22026,22027,22028
0,-70.864000,-68.702000,-65.808000,-61.958000,-60.430000,-62.292000,-67.842000,-72.822000,-76.054000,-76.637000,-74.313000,-69.808000,-65.193000,-65.380000,-67.910000,-66.976000,-65.533000,-61.793000,-57.311000,-59.372000,-63.457000,1.847000,-3.509000,-6.598000,-7.428000,-5.716000,-4.165000,-71.146000,-65.672000,-70.352000,-73.855000,-76.147000,-74.813000,-71.712000,-69.840000,-71.684000,-72.622000,-70.994000,-68.385000,-64.471000,...,-23.583000,-26.165000,-22.390000,-16.063000,-13.442000,-13.513000,-16.834000,-21.169000,-25.142000,-27.067000,-26.478000,-22.257000,-18.800000,-19.059000,-16.779000,-14.062000,-19.177000,-24.592000,-29.303000,-30.549000,-29.563000,-25.992000,-24.749000,-25.923000,-29.975000,-35.225000,-14.989000,-11.806000,-14.140000,-17.141000,-41.104000,-38.093000,-33.270000,-3.021000,2.061000,6.941000,-18.209000,-20.033000,-22.690000,-26.447000
1,166.987000,160.613000,155.388000,149.355000,144.058000,138.986000,137.054000,136.753000,136.775000,136.425000,135.803000,134.573000,132.859000,127.129000,124.717000,126.437000,129.730000,134.869000,138.750000,142.513000,146.986000,-13.863000,-12.016000,-11.104000,-9.965000,-11.891000,-6.562000,163.323000,160.692000,160.900000,165.107000,170.194000,176.722000,180.922000,182.859000,183.440000,179.661000,173.498000,167.139000,168.296000,...,232.433000,227.198000,224.174000,222.321000,221.797000,221.018000,219.880000,216.314000,211.624000,207.678000,203.715000,200.139000,196.995000,200.640000,206.540000,215.610000,215.980000,217.232000,219.510000,223.346000,227.886000,231.733000,232.809000,235.050000,234.997000,230.961000,-4.134000,-6.038000,-3.950000,-0.892000,239.362000,243.693000,246.862000,14.514000,12.979000,12.955000,247.871000,252.021000,257.553000,261.254000
2,93.370000,92.510000,92.400000,91.574000,93.119000,95.600000,96.973000,95.279000,90.971000,85.357000,80.833000,76.570000,75.891000,74.056000,78.263000,84.186000,90.429000,92.158000,88.150000,84.667000,82.390000,-23.148000,-18.902000,-14.744000,-8.349000,-2.138000,1.203000,106.312000,108.485000,110.858000,112.143000,113.139000,112.702000,108.745000,104.983000,98.713000,97.368000,99.884000,101.551000,105.140000,...,17.399000,13.405000,12.436000,13.338000,19.089000,25.371000,30.206000,31.577000,29.679000,26.062000,22.131000,21.171000,24.158000,28.190000,29.155000,16.017000,13.196000,13.227000,16.347000,20.038000,23.992000,24.736000,29.512000,35.283000,39.551000,40.458000,6.633000,2.124000,-3.939000,-7.217000,26.040000,28.161000,27.594000,-4.730000,-6.266000,-10.092000,26.211000,29.360000,29.483000,26.200000
3,-69.557000,-67.297000,-64.016000,-59.714000,-59.905000,-64.723000,-69.992000,-75.083000,-77.579000,-76.539000,-72.477000,-67.331000,-63.538000,-66.034000,-66.100000,-65.604000,-63.537000,-60.029000,-57.718000,-60.703000,-65.576000,-0.335000,-5.843000,-7.606000,-7.935000,-4.714000,-3.414000,-68.531000,-66.657000,-71.323000,-75.114000,-75.333000,-73.385000,-69.167000,-70.494000,-72.674000,-72.678000,-71.234000,-67.966000,-62.222000,...,-24.899000,-25.465000,-19.844000,-14.320000,-13.880000,-14.405000,-19.300000,-23.665000,-27.228000,-27.787000,-24.704000,-19.670000,-18.758000,-17.662000,-15.063000,-15.319000,-21.121000,-26.636000,-30.858000,-31.202000,-27.904000,-23.636000,-23.817000,-26.943000,-32.147000,-36.221000,-12.959000,-12.874000,-16.639000,-15.820000,-39.827000,-36.727000,-30.981000,-1.749000,3.680000,7.869000,-17.005000,-19.525000,-22.658000,-27.406000
4,164.793000,159.269000,153.533000,148.261000,142.517000,139.601000,137.899000,138.001000,138.052000,137.884000,135.238000,135.165000,130.841000,124.572000,125.138000,127.693000,130.523000,135.938000,13

In [ ]:
df1.fillna(0.5, inplace=True)

In [ ]:
##check difference between data transform and without transform -> aslan cherto pert mishe chon y ha dosort nistan!nabayad transform kard be nazaram inja.khodesh dorost dar nazar migire!

In [ ]:
from sklearn.cluster import KMeans 

clusters = 4
  
kmeans = KMeans(n_clusters = clusters) 
kmeans.fit(df1) 
  
l=(kmeans.labels_)
l1=(y_to_label(y))

In [ ]:
d=[]
for i in range(len(l)):
  if l[i] == 0:
    d.append(i)

q , w,e,r=0 , 0,0,0
for i in range(len(d)):
    if l1[d[i]] == 0:
      q = q+1
    if l1[d[i]] == 1:
      w = w+1
    if l1[d[i]] == 2:
      e = e+1
    if l1[d[i]] == 3:
      r = r+1

In [ ]:
q,w,e,r

(73, 64, 70, 63)

In [ ]:

v=[]
for j in range(0,450,30):
  if (j%30)<18:
    v.append(df1[0][j])
v = StandardScaler().fit_transform(v)


ValueError: ignored

## make unit vector